In [2]:
!pip install PyPortfolioOpt
import mlfinlab
import pandas as pd
import yfinance as yf
from mlfinlab.portfolio_optimization.tic import TIC
from datetime import datetime, timedelta
!pip install PyPortfolioOpt
import pypfopt
pypfopt.__version__
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns,plotting
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [73]:
def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31,
        29 if y%4==0 and (not y%100==0 or y%400 == 0) else 28,
        31,30,31,30,31,31,30,31,30,31][m-1])
    return date.replace(day=d,month=m, year=y)

END_DATE = datetime.today() - timedelta(days = 1) #END_DATE is set to yesterday

In [74]:
##For Asset Class

In [75]:

# TLT : iShares 20 Plus Year Treasury Bond ETF
# SPY : SPDR S&P 500 ETF Trust (SPY)
# LQD : iShares iBoxx $ Investment Grade Corporate Bond ETF
# 0P0000P1BG.F : Vanguard Euro Government Bond Index Fund Investor EUR Accumulation
# EXSA.DE : iShares STOXX Europe 600 UCITS ETF
# 0P00000RNA.F : iShares Euro Credit Bond Index Fund 

data = {"Ticker": ["TLT","SPY","LQD","0P0000P1BG.F", "EXSA.DE", "0P00000RNA.F"],
        "Asset Class":[215,225,235,215,225,235],
       "Structure":[2,2,2,2,2,2],}
df = pd.DataFrame(data)





#The numbers in table just represent the category of asset class, They do not matter as long as they are consistent
#Structure column is just added because the code must have 2 columns to evaluate TIC matrix
#Here, Government Bond : Code 15 ; Equity : Code 25; Corporate Bond : Code 35
    

df

,Ticker,Asset Class,Structure
0,TLT,215,2
1,SPY,225,2
2,LQD,235,2
3,0P0000P1BG.F,215,2
4,EXSA.DE,225,2
5,0P00000RNA.F,235,2


In [76]:

data_opt = yf.download("TLT SPY LQD  0P0000P1BG.F EXSA.DE 0P00000RNA.F ", monthdelta(END_DATE,-20), monthdelta(END_DATE,-1))
data_opt = data_opt["Adj Close"]

print(data_opt.head())


[*********************100%***********************]  6 of 6 completed
            0P00000RNA.F  0P0000P1BG.F    EXSA.DE         LQD         SPY  \
Date                                                                        
2019-03-14    214.328003     22.625999  36.191513  110.575577  272.031006   
2019-03-15    214.386993     22.635000  36.487938  110.974419  273.375031   
2019-03-18    214.802002     22.662001  36.550987  110.917458  274.366241   
2019-03-19    214.503006     22.652000  36.764366  110.936440  274.434265   
2019-03-20    214.544998     22.665001  36.424896  111.686646  273.608215   

                   TLT  
Date                    
2019-03-14  117.193222  
2019-03-15  117.968384  
2019-03-18  117.900543  
2019-03-19  117.638939  
2019-03-20  118.869492  


In [77]:
stock_returns =  expected_returns.returns_from_prices(data_opt)

std_ticker = []
for key in stock_returns:
    std_ticker.append(stock_returns[key].std())
print(std_ticker)


[0.0028744594881572744, 0.0017766298789372314, 0.014454646201695188, 0.008576706087767266, 0.017022575578484713, 0.01168503624321027]


In [78]:


# Calculating the empirical correlation matrix
corr_matrix = stock_returns.corr()

print(corr_matrix)

# Calculating the relation of sample length T to the number of variables N
# It's used for de-noising the TIC matrix
tn_relation = stock_returns.shape[0] / stock_returns.shape[1]
print(tn_relation)

# The class that contains the TIC algorithm
tic = TIC()

# Calculating the Theory-Implied Correlation matrix
tic_matrix = pd.DataFrame(tic.tic_correlation(df, corr_matrix, tn_relation, kde_bwidth=0.01))



# Calculating the distance between the empirical and the theory-implied correlation matrices
#matrix_distance = tic.corr_dist(corr_matrix, tic_matrix)
#print(matrix_distance)

              0P00000RNA.F  0P0000P1BG.F   EXSA.DE       LQD       SPY  \
0P00000RNA.F      1.000000      0.769808  0.084248  0.226952  0.018650   
0P0000P1BG.F      0.769808      1.000000  0.100391  0.398753  0.095640   
EXSA.DE           0.084248      0.100391  1.000000  0.254580  0.719942   
LQD               0.226952      0.398753  0.254580  1.000000  0.306316   
SPY               0.018650      0.095640  0.719942  0.306316  1.000000   
TLT               0.421534      0.371338 -0.331287  0.347562 -0.483184   

                   TLT  
0P00000RNA.F  0.421534  
0P0000P1BG.F  0.371338  
EXSA.DE      -0.331287  
LQD           0.347562  
SPY          -0.483184  
TLT           1.000000  
68.16666666666667


In [79]:
##For Country

In [80]:

# TLT : iShares 20 Plus Year Treasury Bond ETF
# SPY : SPDR S&P 500 ETF Trust (SPY)
# LQD : iShares iBoxx $ Investment Grade Corporate Bond ETF
# 0P0000P1BG.F : Vanguard Euro Government Bond Index Fund Investor EUR Accumulation
# EXSA.DE : iShares STOXX Europe 600 UCITS ETF
# 0P00000RNA.F : iShares Euro Credit Bond Index Fund 


data = {"Ticker": ["TLT","SPY","LQD","0P0000P1BG.F", "EXSA.DE", "0P00000RNA.F"],
        "Country":[115,115,115,125,125,125],
       "Structure":[1,1,1,1,1,1]}

#The numbers in table just represent the category of asset class, They do not matter as long as they are consistent
#Structure column is just added because the code must have 2 columns to evaluate TIC matrix
#Here, USA : Code 15 ; Europe : Code 25

df = pd.DataFrame(data)

In [81]:


stock_returns =  expected_returns.returns_from_prices(data_opt)
# Calculating the empirical correlation matrix
corr_matrix = stock_returns.corr()

print(corr_matrix)

# Calculating the relation of sample length T to the number of variables N
# It's used for de-noising the TIC matrix
tn_relation = stock_returns.shape[0] / stock_returns.shape[1]
print(tn_relation)

# The class that contains the TIC algorithm
tic = TIC()

# Calculating the Theory-Implied Correlation matrix
tic_matrix_2 = pd.DataFrame(tic.tic_correlation(df, corr_matrix, tn_relation, kde_bwidth=0.01))



# Calculating the distance between the empirical and the theory-implied correlation matrices
matrix_distance = tic.corr_dist(corr_matrix, tic_matrix)
print(matrix_distance)

              0P00000RNA.F  0P0000P1BG.F   EXSA.DE       LQD       SPY  \
0P00000RNA.F      1.000000      0.769808  0.084248  0.226952  0.018650   
0P0000P1BG.F      0.769808      1.000000  0.100391  0.398753  0.095640   
EXSA.DE           0.084248      0.100391  1.000000  0.254580  0.719942   
LQD               0.226952      0.398753  0.254580  1.000000  0.306316   
SPY               0.018650      0.095640  0.719942  0.306316  1.000000   
TLT               0.421534      0.371338 -0.331287  0.347562 -0.483184   

                   TLT  
0P00000RNA.F  0.421534  
0P0000P1BG.F  0.371338  
EXSA.DE      -0.331287  
LQD           0.347562  
SPY          -0.483184  
TLT           1.000000  
68.16666666666667
0.08327559708171894


In [89]:
big_tic = (tic_matrix+tic_matrix_2)/2
print(big_tic)
print(std_ticker)

          0         1         2         3         4         5
0  1.000000  0.436638  0.207945  0.274845  0.244374  0.414106
1  0.436638  1.000000  0.207905  0.414106  0.244333  0.347241
2  0.207945  0.207905  1.000000  0.192940  0.463349  0.192900
3  0.274845  0.414106  0.192940  1.000000  0.224921  0.394325
4  0.244374  0.244333  0.463349  0.224921  1.000000  0.224881
5  0.414106  0.347241  0.192900  0.394325  0.224881  1.000000
[0.0028744594881572744, 0.0017766298789372314, 0.014454646201695188, 0.008576706087767266, 0.017022575578484713, 0.01168503624321027]


In [94]:
big_tic_cov = pypfopt.risk_models.corr_to_cov(big_tic,std_ticker)*10000
print(std_ticker)
print(big_tic_cov)

[0.0028744594881572744, 0.0017766298789372314, 0.014454646201695188, 0.008576706087767266, 0.017022575578484713, 0.01168503624321027]
          0         1         2         3         4         5
0  0.082625  0.022298  0.086400  0.067759  0.119574  0.139091
1  0.022298  0.031564  0.053391  0.063100  0.073893  0.072087
2  0.086400  0.053391  2.089368  0.239194  1.140095  0.325813
3  0.067759  0.063100  0.239194  0.735599  0.328380  0.395189
4  0.119574  0.073893  1.140095  0.328380  2.897681  0.447309
5  0.139091  0.072087  0.325813  0.395189  0.447309  1.365401
